Here we are going to create a random set of 2000 coordinates and retrieve the city nearest said coordinate along with data on current weather conditions and other weather metrics

In [ ]:
#Import dependencies:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from citipy import citipy
import time
from datetime import datetime
from config import open_weather_key
import requests

In [ ]:
#Generate random set of 2000 latitudes and longitudes:
lats = np.random.uniform(-90,90, size = 2000)
lons = np.random.uniform(-180,180, size = 2000)
lat_lons = zip(lats,lons)  #This will create a list of tuples with each tuple have two elements (lat and lon)
lat_lons


In [ ]:
#Convert lat_lons to a list of coordinates:
coordinates = list(lat_lons)

In [ ]:
#Check list of coordinates:
print(len(coordinates))
print(coordinates[0:10])

In [ ]:
#Get nearest cities and check cities list length
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    if city not in cities:
        cities.append(city)
len(cities)

In [ ]:
cities

In [ ]:
#Make an API call to obtain weather data for each city in list
record_count = 1
set_count = 1
city_data = []
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="
for i, city in enumerate(cities):
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    city_url = url + open_weather_key + "&q=" + city.replace(" ","+")
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count+=1
    try:
        city_weather = requests.get(city_url).json()
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather['weather'][0]['description']
        city_data.append({"City": city.title(),
                          "Country" : city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Weather Description": city_weather_description})
    except KeyError:
        print("City not found. Skipping...")
        pass  
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
city_data

In [ ]:
#Convert the data to a dataframe
city_data_df = pd.DataFrame(city_data)
#Check dataframe structure
city_data_df.head(10)

In [ ]:
#Export the dataframe to a csv file
output_data_file = "WeatherPy_database.csv"
city_data_df.to_csv(output_data_file, index_label = "City_ID")